<h1 align='center'>WebScraping TripAdvisor</h1>

---

El código a continuación tiene por objetivo extraer la **[información solicitada](https://github.com/mozilla/geckodriver/releases/download/v0.28.0/geckodriver-v0.28.0-win64.zip "Word en Google Drive")**, desde la página de **[TripAdvisor](https://www.tripadvisor.cl/Restaurants-g294305-Santiago_Santiago_Metropolitan_Region.html "Web TripAdvisor")** para Ximena. La siguiente celda sólo cumple con el propósito de **silenciar las posibles advertencias** que pudieran levantarse al correr el código, pero no aportan mayormente a la comprensión del proceso por parte del usuario.

In [13]:
%%capture --no-display

import warnings
warnings.filterwarnings('ignore')

La celda anterior asegurará que no se desplieguen advertencias innecesarias para la correcta comprensión y lectura de este informe. A continuación se darán las **instrucciones para instalar las librerías** necesarias para correr el código, cuestión que requiere de un comando para ello, por lo que las instrucciones se despliegan como impresión de una celda.

In [14]:
import os

print(f'Si es la primera vez que corre este programa, por favor abra la terminal PowerShell de Anaconda' +
      f' e ingrese el siguiente comando: "\033[4mpip install -r {os.getcwd()}\\requirements.txt\033[4m"')

Si es la primera vez que corre este programa, por favor abra la terminal PowerShell de Anaconda e ingrese el siguiente comando: "pip install -r C:\Users\nicol\Proyectos\GitHub\Webscraping-TripAdvisor\requirements.txt"


La primera parte fundamental de todo programa, corresponde a la **importación de librerías de Python**. Si acaso hubiera errores en esta primera celda, se aconseja contactar a Nicolás Ganter a su correo: nicolas@ganter.cl

In [15]:
import time
import pickle
import pandas as pd
from tqdm import tqdm
from datetime import datetime

from dask.distributed import Client, progress
import utils

Dentro del código, hay ciertas **variables que es preferible tener en especial consideración**. Entre ellas, encontramos la ubicación del *driver* para *Selenium* que permitirá lanzar una instancia de *Firefox* para navegar la página y extraer los enlaces requeridos en la primera etapa de *webcrawling*. Si aún no ha instalado el driver, acceda a este **[link de descarga](https://github.com/mozilla/geckodriver/releases/download/v0.28.0/geckodriver-v0.28.0-win64.zip "geckodriver download link")**, extraiga el paquete y mueva los documentos a la carpeta de binarios de las librerías de Python.

In [16]:
geckodriver_path = r'C:\Users\nicol\anaconda3\Library\bin\geckodriver'
time_id = datetime.today().strftime('%Y%m%d')
basic_url = 'https://www.tripadvisor.cl'

In [17]:
client = Client()
client

Client Scheduler: tcp://127.0.0.1:56456 Dashboard: http://127.0.0.1:56455/status,Cluster Workers: 3 Cores: 6 Memory: 17.02 GB


<h2>Webcrawling de los restaurantes</h2>
La siguiente celda se encargará de la **extracción de los enlaces** asociados a cada restaurante en las páginas especificadas mediante el enlace de la segunda línea. En este caso, se extraerán los restaurantes de Santiago de Chile. Al final de la celda se imprime la cantidad de restaurantes extraídos, la cantidad de restaurantes disponibles según la página, y el porcentaje capturado por el programa. Nótese que el proceso toma algo así como 10 minutos, por lo que se utilizará un atajo mediante *pickles* (estructura de datos propia de este lenguaje de programación) y se especificará la fecha de captura de la información asociada a éste.

In [18]:
start = time.time()
url = basic_url + '/Restaurants-g294305-Santiago_Santiago_Metropolitan_Region.html'
info = utils.info_restaurants(url, geckodriver_path)

cwd = os.getcwd()
dict_pickles = utils.check_files(dir_files=cwd, keyword='urls')

if len(dict_pickles) == 0:
    urls = utils.gen_pickle(url, geckodriver_path, info['pages'], basic_url, time_id)

else:
    last_pickle = utils.last_pickle(dict_pickles)
    with open(last_pickle, 'rb') as file:
        urls = pickle.load(file)
    
print('Se obtuvieron {} restaurantes de {} lo que corresponde a una extracción del {}%'
      .format(len(urls), info['max_restaurants'], round(len(urls) / info['max_restaurants'] * 100, 2)))

stop = time.time()
print(f'Este proceso tomó {round(stop-start, 2)} segundos en correr.\n')

Información cargada del pickle 20210205_4830_urls.pickle extraído el 05 de febrero del 2021.
Se obtuvieron 4830 restaurantes de 4848 lo que corresponde a una extracción del 99.63%
Este proceso tomó 12.34 segundos en correr.



<h2>Webscraping de los restaurantes</h2>
Con esto concluye la parte más compleja y crítica de la recopilación de enlaces para los restaurantes. No obstante esta tarea continúa luego a nivel de comentarios, **a continuación se procederá a extraer la información solicitada** para cada uno de los restaurantes en la lista. Dado que se utilizan estrategias de computación paralela, no es posible observar el avance, sino abriendo el *Dashboard* cuyo link se encuentra bajo la cuarta celda del código.

In [20]:
start = time.time()
dict_dataframes = utils.check_files(dir_files=cwd, keyword='dataframe')

if len(dict_dataframes) == 0:
    futures = [client.submit(utils.get_restaurant, url_restaurant) for url_restaurant in list(set(urls))]
    results = client.gather(futures)
    
    dict_structure = {'id':[], 'Nombre restaurante':[], 'Promedio de calificaciones':[],
                      'N° de opiniones':[], 'Calificación de viajeros por categoría':[],
                      'Toman medidas de seguridad':[], 'Rankings':[],
                      'Tipo de comida y servicios':[], 'url':[]}
    
    df_restaurants = utils.build_dataframe(dict_structure, results, time_id)
    df_restaurants.to_pickle(f'{time_id}_dataframe_of_{df_restaurants.shape[0]}_restaurants.pickle')
    print(f'Se guardó "{time_id}_dataframe_of_{df_restaurants.shape[0]}_restaurants.pickle" en "{os.getcwd()}".')
    
else:
    last_pickle = utils.last_pickle(dict_dataframes)
    with open(last_pickle, 'rb') as file:
        df_restaurants = pickle.load(file)
    
stop = time.time()
print(f'Este proceso tomó {round(stop-start, 2)} segundos en correr.\n')

Se guardó "20210205_dataframe_of_4830_restaurants.pickle" en "C:\Users\nicol\Proyectos\GitHub\Webscraping-TripAdvisor".
Este proceso tomó 1909.57 segundos en correr.



<h2>Webcrawling de los comentarios</h2>

In [21]:
start = time.time()
dict_files = utils.check_files(dir_files=cwd, keyword='review_urls')

if len(dict_files) == 0:
    futures = [client.submit(utils.review_urls, url_restaurant) for url_restaurant in list(set(urls))]
    results = client.gather(futures)
    
    dict_reviews = {key:value for key, value in results if isinstance(value, list)}
    n_reviews = len(dict_reviews.values())
    
    with open(f'{time_id}_{n_reviews}_review_urls.pickle', 'wb') as file:
        pickle.dump(dict_reviews, file)
    
    print(f'Se guardó "{time_id}_{n_reviews}_review_urls.pickle" en "{os.getcwd()}".')
    
else:
    last_pickle = utils.last_pickle(dict_files)
    with open(last_pickle, 'rb') as file:
        dict_reviews = pickle.load(file)
    
stop = time.time()
print(f'Este proceso tomó {round(stop-start, 2)} segundos en correr.',
      'Se dispone aproximadamente de {} comentarios para extraer.\n'.format(len(dict_reviews.values())*10))

Se guardó "20210205_4321_review_urls.pickle" en "C:\Users\nicol\Proyectos\GitHub\Webscraping-TripAdvisor".
Este proceso tomó 1704.29 segundos en correr. Se dispone aproximadamente de 43210 comentarios para extraer.



<h2>Webscraping de los comentarios</h2>

In [8]:
start = time.time()
dict_files = utils.check_files(dir_files=cwd, keyword='scraped_reviews')
url_reviews = utils.prepare_urls(dict_reviews)

if len(dict_files) == 0:
    futures = [client.submit(utils.get_reviews, url) for url in url_reviews]
    results = client.gather(futures)
    
    dict_structure = {'id':[], 'date_review':[], 'comments':[], 'date_stayed':[], 'response_body':[],
                      'user_name':[], 'user_reviews':[], 'useful_votes':[]}

    df_reviews = utils.build_dataframe(dict_structure, results, time_id)
    df_pathname = f'{time_id}_dataframe_of_{df_reviews.shape[0]}_scraped_reviews.pickle'

    df_reviews.to_pickle(df_pathname)
    print(f'Se guardó "{df_pathname}" en "{os.getcwd()}"')
        
else:
    last_pickle = utils.last_pickle(dict_files)
    df_reviews = pd.read_pickle(last_pickle)

stop = time.time()
print(f'Este proceso tomó {round(stop-start, 2)} segundos en correr.',
      'Se extrajeron {} comentarios.\n'.format(df_reviews.shape[0]))

Se guardó "20210121_dataframe_of_205575_scraped_reviews.pickle" en "C:\Users\nicol\Proyectos\WebScraping TripAdvisor"


,date_review,comments,date_stayed,response_body,user_name,user_reviews,useful_votes
id,,,,,,,
-1.877095e+18,4/08/2017,Fuimos hace una semana con mi novia a conocer ...,08/2017,None,ignaciosaezcl,10,0.0
8.557562e+18,22/11/2016,"A mi hijo mayor le encanta la comida China, es...",10/2016,None,DanaeAchurra,117,0.0
8.557562e+18,26/10/2016,Dentro de esta comuna este restaurantes es muy...,10/2016,None,Carmengloria00,22,0.0
8.557562e+18,22/10/2016,Vez que puedo paso a buscar comida para llevar...,09/2016,None,Pancho89,10,0.0
8.557562e+18,7/01/2020,Enormes platos de comida china. ¡Puedes compar...,12/2019,None,HavBagWiLLTravel,182,0.0
...,...,...,...,...,...,...,...
9.176595e+18,20/07/2018,"La comida es rica, ninguna maravilla. El preci...",10/2017,None,iaguerok,30,0.0
9.176595e+18,22/03/2018,"Sin negar el tamaño de los platos, que llega a...",03/2018,None,Rodrigo M,38,0.0
9.176595e+18,12/12/2017,Fuimos a almorzar con dos compañeros de trabaj...,10/2017,None,LConsoloS,36,0.0


Este proceso tomó 9622.78 segundos en correr. Se extrajeron 205575 comentarios.



<h2>Generación de tablas</h2>

In [13]:
'''
start = time.time()
df_restaurants.to_excel(f'{time_id}_excel_with_{df_restaurants.shape[0]}_restaurants.xlsx')
df_reviews.to_excel(f'{time_id}_excel_with_{df_reviews.shape[0]}_reviews.xlsx')
stop = time.time()

print(f'Este proceso tomó {round(stop-start, 2)} segundos en correr.',
      'Se extrajeron {} comentarios.\n'.format(df_reviews.shape[0]))
'''

In [32]:
'''
import copy

df_restest = df_restaurants.copy()
df_revtest = df_reviews.copy()

id_restaurants = set(df_restest.index.to_list())
id_reviews = set(df_revtest.index.to_list())

print(hash(url_reviews[0]['identifier']), '\n\n', hash(urls[0]))
#.intersection(id_reviews)
'''

6572459385993448040 

 6572459385993448040
